# Impact of humidity and temperature on sensor values

## Setup and config

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import time
import math
from datetime import datetime

plt.rcParams['figure.figsize'] = [14, 4]

#Load SQL file
DATABASE_00 = 'records_00'
DATABASE_01 = 'records_01'
sql_connection = sqlite3.connect(f'{DATABASE_01}.db')

# Measurement groups that are available
GROUPS = ['Capacitive load', 'Digital load', 'Resistive load']
# Measurement group that is to be further analysed in the first sections of the notebook
GROUP = GROUPS[0]

# Threshold in z-score heuristic used for eliminating outlier values in temperature and humidity
ZSCORE_THRESHOLD = 10

# Sample that is further analysed in the first sections of the notebook
SAMPLE = 'sample_29'

# Should regression curves for many of the analysis steps be plotted (might take some seconds per curve)
PLOTREG = False
# Order of the regression curve (works only if PLOTREG is True)
REGRESSION_ORDER = 2

In [ ]:
def datestring_to_timestamp(datestr):
    return time.mktime(datetime.strptime(datestr, '%Y-%m-%d %H:%M:%S.%f').timetuple())

def normalized_mean(df, col, interval):
    md=df[col].groupby(df['date'].dt.to_period(interval)).mean()
    md.index = md.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
    return (md-md.mean())/md.std()

def display_ts_as_date(axis):
    xticks = axis.get_xticks()
    xticks_dates = [datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in xticks]
    axis.set_xticklabels(xticks_dates)

## Load complete Dataset

In [ ]:
#Initial loading of dataset
df_measure_base = pd.read_sql_query('SELECT * FROM Measurements INNER JOIN Fingerprints on Fingerprints.id = Measurements.fingerprint_id INNER JOIN Records on Records.date = Fingerprints.record_date', sql_connection)
df_measure_base

In [ ]:
sql_connection.close()

In [ ]:
def prepare_dataframe(df):
    df_ret = df.copy()
    df_ret = df_ret.drop(['id', 'delta_t', 'record_date'], axis=1)
    df_ret['date_ts'] = df_ret['date'].map(datestring_to_timestamp)
    df_ret['date'] = df_ret['date_ts'].map(datetime.fromtimestamp)
    df_ret['temp_z'] = (df_ret['temperature'] - df_ret['temperature'].mean())/df_ret['temperature'].std(ddof=0)
    df_ret['humid_z'] = (df_ret['humidity'] - df_ret['humidity'].mean())/df_ret['humidity'].std(ddof=0)
    df_ret['date_diff'] = df_ret['date'].diff().map(lambda x: x.total_seconds())
    return df_ret

df_measure_ungrouped = prepare_dataframe(df_measure_base)
df_measure = df_measure_ungrouped.groupby(['component_name']).get_group(GROUP).reset_index(drop=True)

df_measure

In [ ]:
print('Highest temperature outlier:', df_measure[df_measure['temp_z'] == df_measure['temp_z'].max()]['temperature'].tolist()[0], ' zscore: ', df_measure['temp_z'].max())
print('Temperature above Threshold:')
print(df_measure[df_measure['temp_z'] > 10]['temperature'])

In [ ]:
print('Highest humidity outlier: ', df_measure[df_measure['humid_z'] == df_measure['humid_z'].max()]['humidity'].tolist()[0], ' zscore: ', df_measure['humid_z'].max())
print('Humidity above Threshold:')
print(df_measure[df_measure['humid_z'] > 10]['humidity'])

In [ ]:
lst = list(df_measure[[f'sample_{x:02}' for x in range(30)]].iloc[75000])
ax = plt.plot(lst)
plt.show()

## Initial plot of sample

In [ ]:
ax = df_measure.reset_index().plot.scatter(x='date', y=SAMPLE,s=3, alpha=0.2)
plt.show()

In [ ]:

start, end = ('2022-04-01 00:00:00.000000', '2022-04-01 11:59:59.999999')
day_interval = df_measure.copy()[
    (df_measure['date_ts'] > datestring_to_timestamp(start)) &
     (df_measure['date_ts'] < datestring_to_timestamp(end))
]

s_start, s_end = (0, 3)

sample_cols = [f'sample_{i:02}' for i in range(s_start, s_end)]
samples = day_interval[sample_cols]
day_interval[sample_cols] = (samples-samples.mean())/samples.std()

color = iter(cm.rainbow(np.linspace(0, 1, s_end - s_start)))
ax1 = day_interval.reset_index().plot.scatter(x='date', y=sample_cols[0],s=10, alpha=1, label=sample_cols[0])
for sample in sample_cols[1:] :
    day_interval.reset_index().plot.scatter(x='date', y=sample,s=10, alpha=1, ax=ax1, c=next(color).reshape(1,-1), label=sample)

ax.legend(loc=1)
plt.show()



### General data observations

- 30 different samples (`sample_00` - `sample_29`)
- Recorded from `2021-10-05 08:41:07.157388` to `2022-09-09 17:39:00.871733`
    - Consistent data recording starts at `2021-10-29 17:20:53.910281`
- Obvious data anomalies
    - Large increase in values after approx. `2022-01-07 08:58:09.508249`
    - Normalize after approx. `2022-01-21 16:00:22.510406`
    - Larger spread after approx. `2022-05-08 15:42:55.876608` until end of measurement
    - An additional change in values in the middle between `2021-11-12 13:53:21.867046` and `2022-01-07 08:58:09.508249` can also be observed in many samples

In [ ]:
# Plot time ranges
ax = df_measure.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.2)

INTERVALS = [
    ('2021-10-05 08:42:00.000000', '2021-10-29 17:21:00.000000', 'red'),
    ('2021-10-29 17:21:00.000000', '2021-11-12 14:00:00.000000', 'blue'),
    ('2021-11-12 14:00:00.000000', '2021-12-10 14:00:00.000000', 'orange'),
    ('2021-12-10 14:00:00.000000', '2022-01-07 09:00:00.000000', 'cyan'),
    ('2022-01-07 09:00:00.000000', '2022-01-21 16:00:30.000000', 'yellow'),
    ('2022-01-21 17:00:30.000000', '2022-05-08 15:15:00.000000', 'violet'),
    ('2022-05-08 15:15:00.000000', '2022-09-09 17:40:00.000000', 'green')]

def show_intervals(ax, show_intervals=range(0,len(INTERVALS))):
    # Initial missing measurements
    for i in show_intervals:
        interval = INTERVALS[i]
        ax.axvspan(
            datestring_to_timestamp(interval[0]),
            datestring_to_timestamp(interval[1]),
            color=interval[2], alpha=0.2)

#ax.set_ylabel(f"{GROUP} {SAMPLE}")
show_intervals(ax)
display_ts_as_date(ax)
plt.xlabel("date")
plt.savefig(f"overview_{SAMPLE}_{GROUP}_{DATABASE}.svg", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

In [ ]:
for interval in INTERVALS:
    start = datetime.strptime(interval[0], '%Y-%m-%d %H:%M:%S.%f')
    end = datetime.strptime(interval[1], '%Y-%m-%d %H:%M:%S.%f')
    print(start, end, end-start)

## Evaluation of environmental data

In [ ]:
ax1 = df_measure.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.2)

# Add environment information
ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))

#idx = np.array(environment.index.tolist(), dtype='float')

# filtered temperature by zscore to remove outliers
temp_filtered = df_measure[df_measure['temp_z'] < 10]
temp_filtered.plot.scatter(x='date_ts', ax=ax2, y='temperature',s=1, alpha=0.2, c='#922b21')
if PLOTREG:
    sns.regplot(x=temp_filtered['date_ts'], y=temp_filtered['temperature'], ax=ax2, scatter_kws={'s':1, 'color': '#922b21'}, line_kws={"color": "#a93226"}, label='Temperature', order=REGRESSION_ORDER)

# filtered humidity by zscore to remove outliers
humid_filtered = df_measure[df_measure['humid_z'] < 10]
humid_filtered.plot.scatter(x='date_ts', ax=ax2, y='humidity',s=1, alpha=0.2, c='#b9770e')
if PLOTREG:
    sns.regplot(x=humid_filtered['date_ts'], y=humid_filtered['humidity'], ax=ax2, scatter_kws={'s':1, 'color': '#b9770e'}, line_kws={"color": "#d4ac0d"}, label='Humidity', order=REGRESSION_ORDER)

display_ts_as_date(ax2)

ax2.legend(loc=0)
plt.show()

### Environment data observations

- No obvious measurement anomalies in temp and humidity
- Humidity shows a downwards trend
- Temperature shows an upwards trend

## Evaluation of specific interval

In [ ]:
#INTERVAL = ('2022-01-21 18:00:00.000000', '2022-05-07 00:00:00.000000') # Interval between yellow and green
INTERVAL = ('2022-01-21 17:00:30.000000', '2022-05-08 15:15:00.000000') # Inverval marked in violet
#INTERVAL = ('2021-10-29 17:20:53.910281', '2022-09-09 17:39:00.871733') # All data without initial gap

MEAN_INTERVAL = 'D'

df_sample = df_measure[
    (df_measure['date_ts'] > datestring_to_timestamp(INTERVAL[0])) &
     (df_measure['date_ts'] < datestring_to_timestamp(INTERVAL[1]))
]

temp_filtered = df_sample[df_sample['temp_z'] < 10]
humid_filtered = df_sample[df_sample['humid_z'] < 10]

df_sample

In [ ]:
ax1 = df_sample.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.2)

# Add environment information
ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))

if PLOTREG:
    sns.regplot(x=temp_filtered['date_ts'], y=temp_filtered['temperature'], ax=ax2, scatter_kws={'s':1, 'color': '#922b21'}, line_kws={"color": "#a93226"}, label='Temperature', order=REGRESSION_ORDER)
    sns.regplot(x=humid_filtered['date_ts'], y=humid_filtered['humidity'], ax=ax2, scatter_kws={'s':1, 'color': '#b9770e'}, line_kws={"color": "#d4ac0d"}, label='Humidity', order=REGRESSION_ORDER)

xticks = ax2.get_xticks()
xticks_dates = [datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in xticks]
ax2.set_xticklabels(xticks_dates)

ax2.legend(loc=0)
plt.show()


In [ ]:
ax = df_sample.reset_index().plot.scatter(x='date_ts', y='date_diff',s=3, alpha=1)
show_intervals(ax, range(5,6))
plt.show()

In [ ]:
ax1 = df_sample.reset_index().plot.scatter(x='date', y=SAMPLE,s=3, alpha=0.2)

ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))

df_sample[SAMPLE].groupby(df_sample['date'].dt.to_period(MEAN_INTERVAL)).max().plot(kind='line' ,ax=ax2, label='Daily Max')
df_sample[SAMPLE].groupby(df_sample['date'].dt.to_period(MEAN_INTERVAL)).min().plot(kind='line' ,ax=ax2, label='Daily Min')
df_sample[SAMPLE].groupby(df_sample['date'].dt.to_period(MEAN_INTERVAL)).mean().plot(kind='line' ,ax=ax2, label='Daily Mean')

ax2.legend(loc=0)
plt.show()

### Visualisation of Daily Max to Temperature

In [ ]:
mean_temp=temp_filtered['temperature'].groupby(temp_filtered['date'].dt.to_period(MEAN_INTERVAL)).mean()
#mean_temp.index = mean_temp.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
ax1 = mean_temp.plot(kind='line', c='#922b21', label='Temperature')
ax1.set_ylabel("Temperature")
ax2 = ax1.twinx()
mean_humid=humid_filtered['humidity'].groupby(humid_filtered['date'].dt.to_period(MEAN_INTERVAL)).mean()
#mean_humid.index = mean_humid.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
ax2.set_ylabel("Humidity")
mean_humid.plot(kind='line', ax=ax2, c='#b9770e', label='Humidity')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.savefig(f"temp_humid_overview.svg", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

In [ ]:
print("Temperature (Min / Max)", temp_filtered['temperature'].min(), temp_filtered['temperature'].max())
print("Humidity (Min / Max)", temp_filtered['humidity'].min(), temp_filtered['humidity'].max())

In [ ]:
mean_data=df_sample[SAMPLE].groupby(df_sample['date'].dt.to_period(MEAN_INTERVAL)).mean()
mean_data.index = mean_data.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
mean_normalized = (mean_data-mean_data.mean())/mean_data.std()

ax1 = mean_normalized.plot(kind='line', label=f"{SAMPLE} ({GROUP})")
sns.regplot(x=mean_normalized.index, y=mean_normalized, ax=ax1, scatter=False, order=REGRESSION_ORDER, line_kws={"color": "tab:blue"})

mean_temp=temp_filtered['temperature'].groupby(temp_filtered['date'].dt.to_period(MEAN_INTERVAL)).mean()
mean_temp.index = mean_temp.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
mean_temp_normalized = (mean_temp-mean_temp.mean())/mean_temp.std()
mean_temp_normalized.plot(kind='line', ax=ax1, c='#922b21', label='Temperature')
sns.regplot(x=mean_temp_normalized.index, y=mean_temp_normalized, ax=ax1, scatter=False, line_kws={"color": "#a93226"}, order=REGRESSION_ORDER)

mean_humid=humid_filtered['humidity'].groupby(humid_filtered['date'].dt.to_period(MEAN_INTERVAL)).mean()
mean_humid.index = mean_humid.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
mean_humid_normalized = (mean_humid-mean_humid.mean())/mean_humid.std()
mean_humid_normalized.plot(kind='line', ax=ax1, c='#b9770e', label='Humidity')
sns.regplot(x=mean_humid_normalized.index, y=mean_humid_normalized, ax=ax1, scatter=False, line_kws={"color": "#d4ac0d"}, order=REGRESSION_ORDER)

ax1.set_ylabel("Normalized Value")
ax1.legend(loc=1)
show_intervals(ax1, range(5,6))
display_ts_as_date(ax1)

#plt.title('Daily mean sample value compared to temperature and humidity')
plt.savefig(f"mean_temp_humid_{SAMPLE}_{GROUP}_{DATABASE}.svg", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

Correlation coefficients whose magnitude are between 0.7 and 0.9 indicate variables which can be considered highly correlated. Correlation coefficients whose magnitude are between 0.5 and 0.7 indicate variables which can be considered moderately correlated.

In [ ]:
# Correlation between mean sensor and mean temperature
correlations = {c: mean_normalized.corr(mean_temp_normalized, c) for c in ('pearson', 'kendall', 'spearman')}
correlations

In [ ]:
ax1 = df_sample.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=.1, label=f"{SAMPLE}")
ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))

mean_normalized.plot(kind='line', label=f"{SAMPLE} Mean", ax=ax2)
sns.regplot(x=mean_normalized.index, y=mean_normalized, ax=ax2, scatter=False, order=REGRESSION_ORDER, line_kws={"color": "tab:blue"})

mean_temp_normalized_inv = mean_temp_normalized.copy() * -1
mean_temp_normalized_inv.plot(kind='line', ax=ax2, c='#922b21', label='Temperature')
sns.regplot(x=mean_temp_normalized_inv.index, y=mean_temp_normalized_inv, ax=ax2, scatter=False, line_kws={"color": "#a93226"}, order=REGRESSION_ORDER)
#show_intervals(ax1, range(5,6))

#ax1.legend(loc=1)
#ax2.legend(loc=1)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

ax2.set_ylabel("Normalized Value")
display_ts_as_date(ax1)
ax1.set_xlabel("date")

#plt.title(f'Inverse normalized mean daily temperature compared to daily mean sensor value (Spearman Correlation {correlations["spearman"]:.3})')
plt.savefig(f"mean_inv_temp_{SAMPLE}_{GROUP}_{DATABASE}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

In [ ]:
# Correlation between mean sensor and mean humidity
correlations = {c: mean_normalized.corr(mean_humid_normalized, c) for c in ('pearson', 'kendall', 'spearman')}
correlations

In [ ]:
ax1 = mean_normalized.plot(kind='line', label=SAMPLE)
sns.regplot(x=mean_normalized.index, y=mean_normalized, ax=ax1, scatter=False, order=REGRESSION_ORDER, line_kws={"color": "tab:blue"})

mean_humid_normalized.plot(kind='line', ax=ax1, c='#b9770e', label='Humidity')
sns.regplot(x=mean_humid_normalized.index, y=mean_humid_normalized, ax=ax1, scatter=False, line_kws={"color": "#d4ac0d"}, order=REGRESSION_ORDER)

show_intervals(ax1, range(5,6))
ax1.legend(loc=1)

display_ts_as_date(ax1)

#plt.title(f'Normalized mean daily humidity compared to daily mean sensor value (Pearson Correlation {correlations["spearman"]:.3})')
plt.savefig(f"mean_humid_{SAMPLE}_{GROUP}_{DATABASE}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

## Evaluation of Value changes between samples

In [ ]:
SAMPLE_A = 'sample_00'
SAMPLE_B = 'sample_29'

df_samples_w_diff = df_sample.copy()
df_samples_w_diff['diff'] = df_samples_w_diff[SAMPLE_B] - df_samples_w_diff[SAMPLE_A]

ax1 = df_samples_w_diff.plot.scatter(x='date_ts', y='diff',s=3, alpha=.1, label='Sample Difference')

ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))

diff_mean = df_samples_w_diff['diff'].groupby(df_samples_w_diff['date'].dt.to_period(MEAN_INTERVAL)).mean()
diff_mean.index = diff_mean.index.map(lambda x: time.mktime(x.to_timestamp().timetuple()))
#diff_mean.plot(kind='line' ,ax=ax1, label='Mean Daily Diff', c='#2de2f1')

diff_mean_norm = (diff_mean-diff_mean.mean())/diff_mean.std()
diff_mean_norm.plot(kind='line' ,ax=ax2, label='Normalized Difference', c='#42cb1f')

mean_temp_normalized.plot(kind='line', ax=ax2, c='#922b21', label='Temperature')
mean_humid_normalized.plot(kind='line', ax=ax2, c='#b9770e', label='Humidity')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=1)
ax1.legend().remove()
#ax2.legend(loc=1)
ax1.set_ylabel("Sample Difference")
ax2.set_ylabel("Normalized Value")

show_intervals(ax1, range(5,6))
display_ts_as_date(ax1)
ax1.set_xlabel("date")

#plt.title(f'Difference between {SAMPLE_A} and {SAMPLE_B} compared to temperature und humidity daily mean values')
plt.savefig(f"diff_overview_{SAMPLE_A}_{SAMPLE_B}_{GROUP}_{DATABASE}.svg", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

In [ ]:
correlations = {c: diff_mean_norm.corr(mean_temp_normalized, c) for c in ('pearson', 'kendall', 'spearman')}
correlations

In [ ]:
correlations = {c: diff_mean_norm.corr(mean_humid_normalized, c) for c in ('pearson', 'kendall', 'spearman')}
correlations

In [ ]:
ax1 = diff_mean_norm.plot(kind='line', label='Mean Daily Diff Normalized', c='#42cb1f')
mean_temp_normalized.plot(kind='line', ax=ax1, c='#922b21', label='Mean Daily Temperature')
ax1.legend(loc=2)

display_ts_as_date(ax1)

plt.title(f'Difference between {SAMPLE_A} and {SAMPLE_B} compared to inverse temperature daily mean value (Spearman Correleation: {correlations["spearman"]:.5}')
plt.savefig(f"diff_mean_inv_temp_{SAMPLE_A}_{SAMPLE_B}_{GROUP}_{DATABASE}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

# Create overview plots

In [ ]:
GROUPS = ['Capacitive load', 'Digital load', 'Resistive load']

#INTERVAL = ('2021-10-29 17:20:53.910281', '2022-09-09 17:39:00.871733') # All data without initial gap
#INTERVAL = ('2022-05-08 16:00:00.000000', '2022-09-10 00:00:00.000000')
INTERVAL = ('2022-01-21 17:00:30.000000', '2022-05-08 15:15:00.000000') # Inverval marked in violet
MEAN_INTERVAL = 'D'
#SAMPLE = 'sample_13'

SHOW_SCATTER = True

def get_sample_interval(df, interval):
    date_column = df['date_ts'] if 'date_ts' in df else df.index
    return df[
    (date_column > datestring_to_timestamp(interval[0])) &
     (date_column < datestring_to_timestamp(interval[1]))]

df_capacitive = get_sample_interval(df_measure_ungrouped.groupby(['component_name']).get_group('Capacitive load').reset_index(drop=True), INTERVAL)
df_capacitive_mean_sample = normalized_mean(df_capacitive, SAMPLE, MEAN_INTERVAL)

df_digital = get_sample_interval(df_measure_ungrouped.groupby(['component_name']).get_group('Digital load').reset_index(drop=True), INTERVAL)
df_digital_mean_sample = normalized_mean(df_digital, SAMPLE, MEAN_INTERVAL)

df_resistive = get_sample_interval(df_measure_ungrouped.groupby(['component_name']).get_group('Resistive load').reset_index(drop=True), INTERVAL)
df_resistive_mean_sample = normalized_mean(df_resistive, SAMPLE, MEAN_INTERVAL)

# plot first pandas frame in subplot style
#ax1 = df_capacitive_mean_sample.plot(kind='line', label='Capacitive Load Mean Daily')

ax1 = plt.subplot(3, 1, 1)
if SHOW_SCATTER: ax2 = ax1.twinx()
plt.title("Capacitive Load/Temperature")
if SHOW_SCATTER: df_capacitive.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.02,ax=ax1)
df_capacitive_mean_sample.plot(kind='line', label='Capacitive Load Mean Daily', c='tab:cyan',ax=ax2 if SHOW_SCATTER else ax1)
mean_temp_normalized_inv.plot(kind='line', c='#922b21', label='Mean Daily Temperature')
plt.legend(loc=1)
show_intervals(ax1, range(5,6))
display_ts_as_date(ax1)

ax1 = plt.subplot(3, 1, 2)
if SHOW_SCATTER: ax2 = ax1.twinx()
plt.title("Digital Load/Temperature")
if SHOW_SCATTER: df_digital.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.02,ax=ax1)
df_digital_mean_sample.plot(kind='line', label='Digital Load Mean Daily', c='tab:orange', ax=ax2 if SHOW_SCATTER else ax1)
mean_temp_normalized_inv.plot(kind='line', c='#922b21', label='Mean Daily Temperature')
plt.legend(loc=1)
show_intervals(ax1, range(5,6))
display_ts_as_date(ax1)

ax1 = plt.subplot(3, 1, 3)
if SHOW_SCATTER: ax2 = ax1.twinx()
plt.title("Resistive Load/Temperature")
if SHOW_SCATTER: df_resistive.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.02,ax=ax1)
df_resistive_mean_sample.plot(kind='line', label='Resistive Load Mean Daily', c='tab:green',ax=ax2 if SHOW_SCATTER else ax1)
mean_temp_normalized_inv.plot(kind='line', c='#922b21', label='Mean Daily Temperature')
plt.legend(loc=1)
show_intervals(ax1, range(5,6))
display_ts_as_date(ax1)


plt.tight_layout()
plt.savefig(f"comparison_groups_{SAMPLE}_{DATABASE}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

In [ ]:
def split_by_intervals(df):
    df_intervals = []
    for i in INTERVALS:
        df_intervals.append(get_sample_interval(df,i))
    return df_intervals

#SAMPLE='sample_29'
MEAN_INTERVAL = 'D'

fig = plt.figure(constrained_layout=True, figsize=(15, 15))

interval_df_list_cap = split_by_intervals(df_measure_ungrouped.groupby(['component_name']).get_group('Capacitive load').reset_index(drop=True))[1:]
interval_df_list_dig = split_by_intervals(df_measure_ungrouped.groupby(['component_name']).get_group('Digital load').reset_index(drop=True))[1:]
interval_df_list_res = split_by_intervals(df_measure_ungrouped.groupby(['component_name']).get_group('Resistive load').reset_index(drop=True))[1:]

temp_df_list = split_by_intervals(mean_temp_normalized_inv)[1:]
for i,l in enumerate(interval_df_list_cap):
    ax1 = plt.subplot(len(temp_df_list), 1, i+1)
    start = datetime.strptime(INTERVALS[i+1][0], '%Y-%m-%d %H:%M:%S.%f')
    end = datetime.strptime(INTERVALS[i+1][1], '%Y-%m-%d %H:%M:%S.%f')
    plt.title(f'{start} - {end} - Mean daily values - {end - start}')
    ax2 = ax1.twinx()
    normalized_mean(l, SAMPLE, MEAN_INTERVAL).plot(kind='line', label='Capacitive Load', c='tab:cyan',ax=ax2)
    normalized_mean(interval_df_list_dig[i], SAMPLE, MEAN_INTERVAL).plot(kind='line', label='Digital Load', c='tab:orange',ax=ax2)
    normalized_mean(interval_df_list_res[i], SAMPLE, MEAN_INTERVAL).plot(kind='line', label='Resistive Load', c='tab:green',ax=ax2)
    (normalized_mean(l, 'temperature', MEAN_INTERVAL)*-1).plot(kind='line', label='Temperature (Inverted)', c='#922b21',ax=ax2)
    #temp_df_list[i].plot(kind='line', c='#922b21', label='Mean Daily Temperature')
    background = interval_df_list_res[i]
    background.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=0.02,ax=ax1)
    ax2.legend(loc=1)
    show_intervals(ax1, range(i+1,i+2))
    display_ts_as_date(ax2)

#plt.subplots_adjust(left=0.11, bottom=0.07, right=0.95, top=0.95, wspace=0.41, hspace=0.44)
plt.savefig(f"comparison_groups_by_interval_{SAMPLE}_{DATABASE}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
plt.show()

# Compare both record databases

In [ ]:
sql_connection = sqlite3.connect(f'{DATABASE_00}.db')
df_base_00 = pd.read_sql_query('SELECT * FROM Measurements INNER JOIN Fingerprints on Fingerprints.id = Measurements.fingerprint_id INNER JOIN Records on Records.date = Fingerprints.record_date', sql_connection)
df_00 = prepare_dataframe(df_base_00)
sql_connection.close()

sql_connection = sqlite3.connect(f'{DATABASE_01}.db')
df_base_01 = pd.read_sql_query('SELECT * FROM Measurements INNER JOIN Fingerprints on Fingerprints.id = Measurements.fingerprint_id INNER JOIN Records on Records.date = Fingerprints.record_date', sql_connection)
df_01 = prepare_dataframe(df_base_01)


In [ ]:
SAMPLE='sample_29'

for g in GROUPS:
    df_00_grouped = df_00.groupby(['component_name']).get_group(g).reset_index(drop=True)
    df_01_grouped = df_01.groupby(['component_name']).get_group(g).reset_index(drop=True)
    
    ax = df_00_grouped.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=.2,c='tab:blue')
    df_01_grouped.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=.2, ax=ax, c='tab:red')
    
    show_intervals(ax)
    display_ts_as_date(ax)
    plt.savefig(f"databases_comparison_{SAMPLE}_{g}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
    plt.title(f'{g} comparison of both records using sample {SAMPLE}')
    plt.show()


In [ ]:
SAMPLE='sample_00'

for g in GROUPS:
    df_00_grouped = df_00.groupby(['component_name']).get_group(g).reset_index(drop=True)
    df_01_grouped = df_01.groupby(['component_name']).get_group(g).reset_index(drop=True)
    
    ax = df_00_grouped.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=.2,c='tab:blue')
    df_01_grouped.reset_index().plot.scatter(x='date_ts', y=SAMPLE,s=3, alpha=.2, ax=ax, c='tab:red')
    
    show_intervals(ax)
    display_ts_as_date(ax)
    plt.savefig(f"databases_comparison_{SAMPLE}_{g}.png", dpi=300, bbox_inches='tight', pad_inches=.02)
    plt.title(f'{g} comparison of both records using sample {SAMPLE}')
    plt.show()